In [12]:
a = 10
b = 15
c = "안녕"
print(a+b)
print(type(a+b))
print(type(c))

25
<class 'int'>
<class 'str'>


In [13]:
import findspark

In [14]:
findspark.find()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("Spark") \
    .config("soark.sql.rep1.eagerEval.enabled", True) \
    .getOrCreate()

In [15]:
data=[("Z", 1),("A", 20),("B", 30),("C", 40),("B", 30),("B", 60)]
inputRDD = spark.sparkContext.parallelize(data)  # 타입RDD로 변경
print(inputRDD)
inputRDD.collect() # collect : spark에서 print문과 같음
py_rdd = inputRDD.collect()
for i in py_rdd:
   print(i[0] + "," + str(i[1]))

ParallelCollectionRDD[46] at readRDDFromFile at PythonRDD.scala:287
Z,1
A,20
B,30
C,40
B,30
B,60


In [16]:
listRdd = spark.sparkContext.parallelize([1,2,3,4,5,3,2])
listRdd.collect()

print("Count : "+str(listRdd.count()))

print("countByValue :  "+str(listRdd.countByValue()))

Count : 7
countByValue :  defaultdict(<class 'int'>, {1: 1, 2: 2, 3: 2, 4: 1, 5: 1})


In [17]:
inputRDD.collect()
listRdd.collect()

print("first :  "+str(listRdd.first()))
print("first :  "+str(inputRDD.first()))

first :  1
first :  ('Z', 1)


In [18]:
inputRDD.collect()
listRdd.collect()

print("top : "+str(listRdd.top(2)))

print("top : "+str(inputRDD.top(2)))

print("min :  "+str(listRdd.min()))

print("min :  "+str(inputRDD.min()))

print("max :  "+str(listRdd.max()))

print("max :  "+str(inputRDD.max()))

top : [5, 4]
top : [('Z', 1), ('C', 40)]
min :  1
min :  ('A', 20)
max :  5
max :  ('Z', 1)


In [19]:
sc = spark.sparkContext
rdd = sc.parallelize(['a,b,c','d,e,f','g,h,i'])

map_rdd = rdd.map(lambda x: x.split(','))
print(map_rdd.collect())

flatmap_rdd = rdd.flatMap(lambda x: x.split(','))
print(flatmap_rdd.collect())

[['a', 'b', 'c'], ['d', 'e', 'f'], ['g', 'h', 'i']]
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']


In [20]:
data=[("Z", 1),("A", 20),("B", 30),("C", 40),("B", 30),("B", 60)]
inputRDD = spark.sparkContext.parallelize(data)

rdd2=inputRDD.reduceByKey(lambda a,b: a+b)

rdd2.collect()

for element in rdd2.collect():
    print(element)

('Z', 1)
('A', 20)
('B', 120)
('C', 40)


In [21]:
# 23.08.31
columns = ["language","users_count"]
data = [("Java", "20000"), ("Python", "100000"), ("Scala", "3000")]

rdd = spark.sparkContext.parallelize(data)

dfFromRDD = rdd.toDF()
dfFromRDD.printSchema()

dfFromRDD1 = rdd.toDF(columns)
dfFromRDD1.printSchema()

dfFromRDD2 = spark.createDataFrame(rdd).toDF(*columns)
dfFromRDD2.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: string (nullable = true)

root
 |-- language: string (nullable = true)
 |-- users_count: string (nullable = true)

root
 |-- language: string (nullable = true)
 |-- users_count: string (nullable = true)



In [22]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
data2 = [("James","","Smith","36636","M",3000),
    ("Michael","Rose","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Maria","Anne","Jones","39192","F",4000),
    ("Jen","Mary","Brown","","F",-1)
]

schema = StructType([ 
    StructField("firstname",StringType(),True), 
    StructField("middlename",StringType(),True), 
    StructField("lastname",StringType(),True), 
    StructField("id", StringType(), True), 
    StructField("gender", StringType(), True), 
    StructField("salary", IntegerType(), True)
  ])
 
df = spark.createDataFrame(data=data2,schema=schema)
df.printSchema()
df.show(truncate=False)

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|id   |gender|salary|
+---------+----------+--------+-----+------+------+
|James    |          |Smith   |36636|M     |3000  |
|Michael  |Rose      |        |40288|M     |4000  |
|Robert   |          |Williams|42114|M     |4000  |
|Maria    |Anne      |Jones   |39192|F     |4000  |
|Jen      |Mary      |Brown   |     |F     |-1    |
+---------+----------+--------+-----+------+------+



In [23]:
df2 = spark.read.csv("./simple_zipcodes.csv")
df2.show(truncate=False)
df2 = spark.read.text("./simple_zipcodes.txt")
df2.show(truncate=False)
df2 = spark.read.json("./simple_zipcodes.json")
df2.show(truncate=False)

+---+----------------------------------------------------------------------------------+
|_c0|_c1                                                                               |
+---+----------------------------------------------------------------------------------+
|Id |JsonValue                                                                         |
|1  |{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}        |
|2  |{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PASEO COSTA DEL SUR","State":"PR"}|
|3  |{"Zipcode":709,"ZipCodeType":"STANDARD","City":"BDA SAN LUIS","State":"PR"}       |
|4  |{"Zipcode":76166,"ZipCodeType":"UNIQUE","City":"CINGULAR WIRELESS","State":"TX"}  |
|5  |{"Zipcode":76177,"ZipCodeType":"STANDARD","City":"FORT WORTH","State":"TX"}       |
|6  |{"Zipcode":76177,"ZipCodeType":"STANDARD","City":"FT WORTH","State":"TX"}         |
|7  |{"Zipcode":704,"ZipCodeType":"STANDARD","City":"URB EUGENE RICE","State":"PR"}    |
|8  |{"Zipcode":85209

In [24]:
emptyRDD = spark.sparkContext.emptyRDD()
print(emptyRDD)
rdd2= spark.sparkContext.parallelize([])
print(rdd2)

EmptyRDD[114] at emptyRDD at NativeMethodAccessorImpl.java:0
ParallelCollectionRDD[115] at readRDDFromFile at PythonRDD.scala:287


In [25]:
# p.50 DataFrame 생성 / StructType
from pyspark.sql.types import StructType,StructField, StringType
schema = StructType([
  StructField('firstname', StringType(), True),
  StructField('middlename', StringType(), True),
  StructField('lastname', StringType(), True)
])
df = spark.createDataFrame(emptyRDD,schema)
df.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)



In [26]:
# p.51 emptyRDD
# to.DF함수, 함수 내 스키마 구조 지정
df1 = emptyRDD.toDF(schema)
df1.printSchema()

# 첫번째 인자: [], 두번째 인자: 생성했던 스키마 구조
df2 = spark.createDataFrame([], schema)
df2.printSchema()

# 첫번째 인자: [], 두번째 인자: 비어있는 구조
df3 = spark.createDataFrame([], StructType([]))
df3.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)

root



In [27]:
# p.52 RDD to Dataframe
# parallelize()를 통해 RDD 형태 데이터 생성
dept = [("Finance",10),("Marketing",20),("Sales",30),("IT",40)]
rdd = spark.sparkContext.parallelize(dept)
deptColumns = ["dept_name","dept_id"]
df2 = rdd.toDF(deptColumns) # 변수.toDF("컬럼명 변수"):Dataframe생성
df2.printSchema()
df2.show(truncate=False)

root
 |-- dept_name: string (nullable = true)
 |-- dept_id: long (nullable = true)

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|Finance  |10     |
|Marketing|20     |
|Sales    |30     |
|IT       |40     |
+---------+-------+



In [28]:
# p.53 첫번째 인자: RDD변수, 두번째 인자:컬럼명 변수 -> Dataframe생성
deptDF = spark.createDataFrame(rdd, deptColumns)
deptDF.printSchema()
deptDF.show(truncate=False)

root
 |-- dept_name: string (nullable = true)
 |-- dept_id: long (nullable = true)

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|Finance  |10     |
|Marketing|20     |
|Sales    |30     |
|IT       |40     |
+---------+-------+



In [29]:
# p.54 두번째 인자: StructType로 구성한 구조
from pyspark.sql.types import StructType,StructField, StringType
deptSchema = StructType([       
    StructField('dept_name', StringType(), True),
    StructField('dept_id', StringType(), True)
])
deptDF1 = spark.createDataFrame(rdd, schema = deptSchema)
deptDF1.printSchema()
deptDF1.show(truncate=False)

root
 |-- dept_name: string (nullable = true)
 |-- dept_id: string (nullable = true)

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|Finance  |10     |
|Marketing|20     |
|Sales    |30     |
|IT       |40     |
+---------+-------+



In [30]:
# p.58 
# Dataframe 생성
dept = [("Finance",10),("Marketing",20),("Sales",30),("IT",40)]
rdd = spark.sparkContext.parallelize(dept)
deptColumns = ["dept_name","dept_id"]
df2 = rdd.toDF(deptColumns)
df2.printSchema()
df2.show(truncate=False)

# Python Dataframe으로 변경
import pandas
pandasDF = df2.toPandas()
print(pandasDF)

root
 |-- dept_name: string (nullable = true)
 |-- dept_id: long (nullable = true)

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|Finance  |10     |
|Marketing|20     |
|Sales    |30     |
|IT       |40     |
+---------+-------+

   dept_name  dept_id
0    Finance       10
1  Marketing       20
2      Sales       30
3         IT       40


In [31]:
# p.59 PySpark의 Dataframe으로 변경
PySparkDF = spark.createDataFrame(pandasDF)
PySparkDF.show(truncate=False)

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|Finance  |10     |
|Marketing|20     |
|Sales    |30     |
|IT       |40     |
+---------+-------+



In [32]:
# p.60 show함수
from pyspark.sql import SparkSession
columns = ["Seqno","Quote"]
data = [("1", "Be the change that you wish to see in the world"),
    ("2", "Everyone thinks of changing the world, but no one thinks of changing himself."),
    ("3", "The purpose of our lives is to be happy."),
    ("4", "Be cool.")]
df = spark.createDataFrame(data,columns)
df.show()

+-----+--------------------+
|Seqno|               Quote|
+-----+--------------------+
|    1|Be the change tha...|
|    2|Everyone thinks o...|
|    3|The purpose of ou...|
|    4|            Be cool.|
+-----+--------------------+



In [33]:
# p.61
df.show(truncate=False) # 전체
df.show(2,truncate=False) # 원하는 행, 전체
df.show(2,truncate=25) # 원하는 행, 정수값만큼
df.show(n=3,truncate=25,vertical=True) # 수직 표현

+-----+-----------------------------------------------------------------------------+
|Seqno|Quote                                                                        |
+-----+-----------------------------------------------------------------------------+
|1    |Be the change that you wish to see in the world                              |
|2    |Everyone thinks of changing the world, but no one thinks of changing himself.|
|3    |The purpose of our lives is to be happy.                                     |
|4    |Be cool.                                                                     |
+-----+-----------------------------------------------------------------------------+

+-----+-----------------------------------------------------------------------------+
|Seqno|Quote                                                                        |
+-----+-----------------------------------------------------------------------------+
|1    |Be the change that you wish to see in the worl

In [34]:
# p.62 Column
data=[("James",23),("Ann",40)]
df=spark.createDataFrame(data).toDF("name.fname","gender")
df.printSchema()
df.select(df.gender).show()
df.select(df["gender"]).show()
df.select(df["`name.fname`"]).show()

root
 |-- name.fname: string (nullable = true)
 |-- gender: long (nullable = true)

+------+
|gender|
+------+
|    23|
|    40|
+------+

+------+
|gender|
+------+
|    23|
|    40|
+------+

+----------+
|name.fname|
+----------+
|     James|
|       Ann|
+----------+



In [35]:
# p.63 col()함수
from pyspark.sql.functions import col
df.select(col("gender")).show()
df.select(col("`name.fname`")).show()

+------+
|gender|
+------+
|    23|
|    40|
+------+

+----------+
|name.fname|
+----------+
|     James|
|       Ann|
+----------+



In [36]:
# p.64 Row()함수 이용 데이터 생성
# Row():컬럼명과 해당값, 중첩컬럼데이터구성가능
from pyspark.sql import Row
data=[Row(name="James",prop=Row(hair="black",eye="blue")),
      Row(name="Ann",prop=Row(hair="grey",eye="black"))]
df=spark.createDataFrame(data)
df.printSchema()
df.show()

root
 |-- name: string (nullable = true)
 |-- prop: struct (nullable = true)
 |    |-- hair: string (nullable = true)
 |    |-- eye: string (nullable = true)

+-----+-------------+
| name|         prop|
+-----+-------------+
|James|{black, blue}|
|  Ann|{grey, black}|
+-----+-------------+



In [37]:
# p.65 
# .:중첩구조 데이터에 접근, *:중첩구조의 컬럼 풀어서 반환
df.select(df.prop.hair).show()
df.select(df["prop.hair"]).show()
df.select(col("prop.hair")).show()
df.select(col("prop.*")).show()

+---------+
|prop.hair|
+---------+
|    black|
|     grey|
+---------+

+-----+
| hair|
+-----+
|black|
| grey|
+-----+

+-----+
| hair|
+-----+
|black|
| grey|
+-----+

+-----+-----+
| hair|  eye|
+-----+-----+
|black| blue|
| grey|black|
+-----+-----+



In [38]:
# p.66,67 컬럼의 연산 및 비교
# 비교 연산자(<,>,==) -> Boolean을 반환
data=[(100,2,1),(200,3,4),(300,4,4)]
df=spark.createDataFrame(data).toDF("col1","col2","col3")
df.show()

df.select(df.col1 + df.col2).show()
df.select(df.col1 - df.col2).show() 
df.select(df.col1 * df.col2).show()
df.select(df.col1 / df.col2).show() 
df.select(df.col1 % df.col2).show() # 나머지
df.select(df.col2 > df.col3).show()
df.select(df.col2 < df.col3).show()
df.select(df.col2 == df.col3).show()

+----+----+----+
|col1|col2|col3|
+----+----+----+
| 100|   2|   1|
| 200|   3|   4|
| 300|   4|   4|
+----+----+----+

+-------------+
|(col1 + col2)|
+-------------+
|          102|
|          203|
|          304|
+-------------+

+-------------+
|(col1 - col2)|
+-------------+
|           98|
|          197|
|          296|
+-------------+

+-------------+
|(col1 * col2)|
+-------------+
|          200|
|          600|
|         1200|
+-------------+

+-----------------+
|    (col1 / col2)|
+-----------------+
|             50.0|
|66.66666666666667|
|             75.0|
+-----------------+

+-------------+
|(col1 % col2)|
+-------------+
|            0|
|            2|
|            0|
+-------------+

+-------------+
|(col2 > col3)|
+-------------+
|         true|
|        false|
|        false|
+-------------+

+-------------+
|(col2 < col3)|
+-------------+
|        false|
|         true|
|        false|
+-------------+

+-------------+
|(col2 = col3)|
+-------------+
|        fals

In [39]:
# p.68,69
data=[("James","Bond","100",None),
      ("Ann","Varsa","200",'F'),
      ("Tom Cruise","XXX","400",''),
      ("Tom Brand",None,"400",'M')] 
columns=["fname","lname","id","gender"]
df=spark.createDataFrame(data,columns)
df.show()

# alias 함수 사용하여 별칭이용
df.select(df.fname.alias("first_name"), df.lname.alias("last_name")).show()

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|     James| Bond|100|  null|
|       Ann|Varsa|200|     F|
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
+----------+-----+---+------+

+----------+---------+
|first_name|last_name|
+----------+---------+
|     James|     Bond|
|       Ann|    Varsa|
|Tom Cruise|      XXX|
| Tom Brand|     null|
+----------+---------+



In [40]:
# p.70 expr():컬럼 합쳐서 표현
from pyspark.sql.functions import expr
df.select(expr(" fname ||','|| lname").alias("fullName")).show()

+--------------+
|      fullName|
+--------------+
|    James,Bond|
|     Ann,Varsa|
|Tom Cruise,XXX|
|          null|
+--------------+



In [41]:
# p.71 asc:오름차순, desc:내림차순
df.sort(df.fname.asc()).show()
df.sort(df.fname.desc()).show()

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|       Ann|Varsa|200|     F|
|     James| Bond|100|  null|
| Tom Brand| null|400|     M|
|Tom Cruise|  XXX|400|      |
+----------+-----+---+------+

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
|     James| Bond|100|  null|
|       Ann|Varsa|200|     F|
+----------+-----+---+------+



In [42]:
# p.72 cast: 형변환
df.printSchema()
df.select(df.fname,df.id.cast("int")).printSchema()

root
 |-- fname: string (nullable = true)
 |-- lname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)

root
 |-- fname: string (nullable = true)
 |-- id: integer (nullable = true)



In [43]:
# p.73 between: 범위, contains: 특정 문자열 포함행 출력
# ==은 풀 문자열 검색 수행
df.filter(df.id.between(100,300)).show()
df.filter(df.fname.contains("Cruise")).show()
df.filter(df.fname == "Cruise").show()
df.filter(df.fname == "Tom Cruise").show()

+-----+-----+---+------+
|fname|lname| id|gender|
+-----+-----+---+------+
|James| Bond|100|  null|
|  Ann|Varsa|200|     F|
+-----+-----+---+------+

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|Tom Cruise|  XXX|400|      |
+----------+-----+---+------+

+-----+-----+---+------+
|fname|lname| id|gender|
+-----+-----+---+------+
+-----+-----+---+------+

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|Tom Cruise|  XXX|400|      |
+----------+-----+---+------+



In [44]:
# p.74 startswith: 특정문자 시작되는 행 검색, endswith: 특정문자 끝부터 시작하는 행 검색
df.filter(df.fname.startswith("T")).show()
df.filter(df.fname.endswith("Cruise")).show()

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
+----------+-----+---+------+

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|Tom Cruise|  XXX|400|      |
+----------+-----+---+------+



In [45]:
# p.75 isNull: null만 출력, isNotNull: null 제외출력
df.filter(df.lname.isNull()).show()
df.filter(df.lname.isNotNull()).show()

+---------+-----+---+------+
|    fname|lname| id|gender|
+---------+-----+---+------+
|Tom Brand| null|400|     M|
+---------+-----+---+------+

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|     James| Bond|100|  null|
|       Ann|Varsa|200|     F|
|Tom Cruise|  XXX|400|      |
+----------+-----+---+------+



In [46]:
# p.76 substr:특정컬럼에 원하는 문자열 만큼
# 문자열의 첫번째부터 두번째까지
df.select(df.fname.substr(1,2).alias("substr")).show()

+------+
|substr|
+------+
|    Ja|
|    An|
|    To|
|    To|
+------+



In [47]:
# p.77 when(): 다양한 조건들을 순서적으로 추가
# otherwise(): 그외의 조건
df.show()

from pyspark.sql.functions import when
df.select(df.fname,df.lname,when(df.gender=="M","Male")
.when(df.gender=="F","Female")
.alias("new_gender")).show()

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|     James| Bond|100|  null|
|       Ann|Varsa|200|     F|
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
+----------+-----+---+------+

+----------+-----+----------+
|     fname|lname|new_gender|
+----------+-----+----------+
|     James| Bond|      null|
|       Ann|Varsa|    Female|
|Tom Cruise|  XXX|      null|
| Tom Brand| null|      Male|
+----------+-----+----------+



In [48]:
# p.78 isin:특정 값 존재 행 출력
li=["100","200"]
df.select(df.fname,df.lname,df.id).filter(df.id.isin(li)).show()

+-----+-----+---+
|fname|lname| id|
+-----+-----+---+
|James| Bond|100|
|  Ann|Varsa|200|
+-----+-----+---+



In [49]:
# p.79 Select
# p.80 [:] : 원하는 컬럼의 길이만큼 출력(열의길이)
df.show()
df.select(df.columns[:3]).show(3)
df.select(df.columns[2:4]).show(3)

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|     James| Bond|100|  null|
|       Ann|Varsa|200|     F|
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
+----------+-----+---+------+

+----------+-----+---+
|     fname|lname| id|
+----------+-----+---+
|     James| Bond|100|
|       Ann|Varsa|200|
|Tom Cruise|  XXX|400|
+----------+-----+---+
only showing top 3 rows

+---+------+
| id|gender|
+---+------+
|100|  null|
|200|     F|
|400|      |
+---+------+
only showing top 3 rows



In [50]:
# p.81 withColumn: 컬럼 생성 or 변경, cast: 형변환
from pyspark.sql.functions import col
df2 = df.withColumn("id",col("id").cast("Integer"))
df2.printSchema()
df.printSchema()

root
 |-- fname: string (nullable = true)
 |-- lname: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)

root
 |-- fname: string (nullable = true)
 |-- lname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)



In [51]:
# p.82 col()에 연산 수행
df3 = df.withColumn("id",col("id")*100)
df3.show()
df.show() 

+----------+-----+-------+------+
|     fname|lname|     id|gender|
+----------+-----+-------+------+
|     James| Bond|10000.0|  null|
|       Ann|Varsa|20000.0|     F|
|Tom Cruise|  XXX|40000.0|      |
| Tom Brand| null|40000.0|     M|
+----------+-----+-------+------+

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|     James| Bond|100|  null|
|       Ann|Varsa|200|     F|
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
+----------+-----+---+------+



In [52]:
# p.83 새로운 컬럼 생성
df4 = df.withColumn("new_id",col("id")* -1)
df4.show()
df.show()

+----------+-----+---+------+------+
|     fname|lname| id|gender|new_id|
+----------+-----+---+------+------+
|     James| Bond|100|  null|-100.0|
|       Ann|Varsa|200|     F|-200.0|
|Tom Cruise|  XXX|400|      |-400.0|
| Tom Brand| null|400|     M|-400.0|
+----------+-----+---+------+------+

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|     James| Bond|100|  null|
|       Ann|Varsa|200|     F|
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
+----------+-----+---+------+



In [53]:
# p.84 withColumnRenamed: 컬럼명 변경(기존컬럼명,변경컬럼명)
df5 = df.withColumnRenamed("id","new_id")
df5.show()
df.show()

+----------+-----+------+------+
|     fname|lname|new_id|gender|
+----------+-----+------+------+
|     James| Bond|   100|  null|
|       Ann|Varsa|   200|     F|
|Tom Cruise|  XXX|   400|      |
| Tom Brand| null|   400|     M|
+----------+-----+------+------+

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|     James| Bond|100|  null|
|       Ann|Varsa|200|     F|
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
+----------+-----+---+------+



In [54]:
# p.85 다수의 컬럼명 변경 : .을 이용하여 계속적으로 함수 사용
df.show()
df.withColumnRenamed("id","new_id").withColumnRenamed("fname", "new_fname").show()

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|     James| Bond|100|  null|
|       Ann|Varsa|200|     F|
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
+----------+-----+---+------+

+----------+-----+------+------+
| new_fname|lname|new_id|gender|
+----------+-----+------+------+
|     James| Bond|   100|  null|
|       Ann|Varsa|   200|     F|
|Tom Cruise|  XXX|   400|      |
| Tom Brand| null|   400|     M|
+----------+-----+------+------+



In [55]:
# p.86 모든 컬럼명 변경
# 새로운 컬럼명 리스트 생성 -> toDF의 *: 모든 값을 가져오겠다는 의미
newColumns = ["newCol1","newCol2","newCol3","newCol4"]
new_df = df.toDF(*newColumns)
new_df.show()
df.show()

+----------+-------+-------+-------+
|   newCol1|newCol2|newCol3|newCol4|
+----------+-------+-------+-------+
|     James|   Bond|    100|   null|
|       Ann|  Varsa|    200|      F|
|Tom Cruise|    XXX|    400|       |
| Tom Brand|   null|    400|      M|
+----------+-------+-------+-------+

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|     James| Bond|100|  null|
|       Ann|Varsa|200|     F|
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
+----------+-----+---+------+



In [56]:
# p.87 drop: 컬럼 삭제
df6 = df.drop("gender")
df6.show()
df.show()

+----------+-----+---+
|     fname|lname| id|
+----------+-----+---+
|     James| Bond|100|
|       Ann|Varsa|200|
|Tom Cruise|  XXX|400|
| Tom Brand| null|400|
+----------+-----+---+

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|     James| Bond|100|  null|
|       Ann|Varsa|200|     F|
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
+----------+-----+---+------+



In [58]:
# 23.09.01
# p.88 filter(): 원하는 데이터 출력
# 일치하지 않음: !=, ~(...==...)
df.show()
df.filter(df.fname == "James").show()
df.filter(df.fname != "James").show() 
df.filter(~(df.fname == "James")).show()

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|     James| Bond|100|  null|
|       Ann|Varsa|200|     F|
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
+----------+-----+---+------+

+-----+-----+---+------+
|fname|lname| id|gender|
+-----+-----+---+------+
|James| Bond|100|  null|
+-----+-----+---+------+

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|       Ann|Varsa|200|     F|
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
+----------+-----+---+------+

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|       Ann|Varsa|200|     F|
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
+----------+-----+---+------+



In [60]:
# p.89
df.show()
df.filter("gender == 'M'").show()
df.filter("gender != 'M'").show()
df.filter("gender <> 'M'").show()

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|     James| Bond|100|  null|
|       Ann|Varsa|200|     F|
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
+----------+-----+---+------+

+---------+-----+---+------+
|    fname|lname| id|gender|
+---------+-----+---+------+
|Tom Brand| null|400|     M|
+---------+-----+---+------+

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|       Ann|Varsa|200|     F|
|Tom Cruise|  XXX|400|      |
+----------+-----+---+------+

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|       Ann|Varsa|200|     F|
|Tom Cruise|  XXX|400|      |
+----------+-----+---+------+



In [61]:
# p.90 AND/OR/NOT 연산
df.show()
df.filter((df.gender == "F") & (df.fname == "Ann")).show(truncate=False) 

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|     James| Bond|100|  null|
|       Ann|Varsa|200|     F|
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
+----------+-----+---+------+

+-----+-----+---+------+
|fname|lname|id |gender|
+-----+-----+---+------+
|Ann  |Varsa|200|F     |
+-----+-----+---+------+



In [62]:
# p.91 like(): 원하는 문자열이 포함된 데이터 조회
df.show()
df.filter(df.fname.like("Tom%")).show()

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|     James| Bond|100|  null|
|       Ann|Varsa|200|     F|
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
+----------+-----+---+------+

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
+----------+-----+---+------+



In [65]:
# p.92
data = [("James", "Sales", 3000),
  ("Robert", "Sales", 4100),
  ("Maria", "Finance", 3000),
  ("James", "Sales", 3000),
  ("Scott", "Finance", 3000),
  ("Jen", "Finance", 3900),
  ("Maria", "Finance", 3000),
]
columns= ["employee_name", "department", "salary"]
df = spark.createDataFrame(data = data, schema = columns)
df.printSchema()
df.show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|James        |Sales     |3000  |
|Robert       |Sales     |4100  |
|Maria        |Finance   |3000  |
|James        |Sales     |3000  |
|Scott        |Finance   |3000  |
|Jen          |Finance   |3900  |
|Maria        |Finance   |3000  |
+-------------+----------+------+



In [67]:
# p.93 Distinct(),dropDuplicates() : 중복제거
distinctDF = df.distinct()
print("Distinct count: "+str(distinctDF.count()))
distinctDF.show(truncate=False)

df2 = df.dropDuplicates()
print("Distinct count: "+str(df2.count()))
df2.show(truncate=False)

Distinct count: 5
+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|Jen          |Finance   |3900  |
|Scott        |Finance   |3000  |
|James        |Sales     |3000  |
|Robert       |Sales     |4100  |
|Maria        |Finance   |3000  |
+-------------+----------+------+

Distinct count: 5
+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|Jen          |Finance   |3900  |
|Scott        |Finance   |3000  |
|James        |Sales     |3000  |
|Robert       |Sales     |4100  |
|Maria        |Finance   |3000  |
+-------------+----------+------+



In [68]:
# p.94 dropDuplicates() : 중복을 제외하고 싶은 컬럼 다수 지정 가능
df.show()
dropDisDF = df.dropDuplicates(["department","salary"])
print("Distinct count of department & salary : "+str(dropDisDF.count()))
dropDisDF.show(truncate=False)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|        James|     Sales|  3000|
|        Scott|   Finance|  3000|
|          Jen|   Finance|  3900|
|        Maria|   Finance|  3000|
+-------------+----------+------+

Distinct count of department & salary : 4
+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|Maria        |Finance   |3000  |
|Jen          |Finance   |3900  |
|James        |Sales     |3000  |
|Robert       |Sales     |4100  |
+-------------+----------+------+



In [69]:
# p.95 Sort:컬럼의 데이터 정렬
df.show()
df.sort("department","salary").show()
df.sort(col("department"),col("salary")).show()

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|        James|     Sales|  3000|
|        Scott|   Finance|  3000|
|          Jen|   Finance|  3900|
|        Maria|   Finance|  3000|
+-------------+----------+------+

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        Maria|   Finance|  3000|
|        Scott|   Finance|  3000|
|        Maria|   Finance|  3000|
|          Jen|   Finance|  3900|
|        James|     Sales|  3000|
|        James|     Sales|  3000|
|       Robert|     Sales|  4100|
+-------------+----------+------+

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        Maria|   Finance|  3000|
|        Scott|   Finance|  3000|
|        Maria|   Finance|  3000|
|          Jen|   Finance|  3900|
|        Jam

In [72]:
# p.96 orderBy: 데이터 정렬
df.show()
df.orderBy("department","salary").show()
df.orderBy(col("department"),col("salary")).show()

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|        James|     Sales|  3000|
|        Scott|   Finance|  3000|
|          Jen|   Finance|  3900|
|        Maria|   Finance|  3000|
+-------------+----------+------+

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        Maria|   Finance|  3000|
|        Scott|   Finance|  3000|
|        Maria|   Finance|  3000|
|          Jen|   Finance|  3900|
|        James|     Sales|  3000|
|        James|     Sales|  3000|
|       Robert|     Sales|  4100|
+-------------+----------+------+

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        Maria|   Finance|  3000|
|        Scott|   Finance|  3000|
|        Maria|   Finance|  3000|
|          Jen|   Finance|  3900|
|        Jam

In [73]:
# p.97 asc:오름차순, desc:내림차순
df.sort(df.department.asc(),df.salary.desc()).show()
df.sort(col("department").asc(),col("salary").desc()).show()
df.orderBy(col("department").asc(),col("salary").desc()).show()

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|          Jen|   Finance|  3900|
|        Maria|   Finance|  3000|
|        Scott|   Finance|  3000|
|        Maria|   Finance|  3000|
|       Robert|     Sales|  4100|
|        James|     Sales|  3000|
|        James|     Sales|  3000|
+-------------+----------+------+

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|          Jen|   Finance|  3900|
|        Maria|   Finance|  3000|
|        Scott|   Finance|  3000|
|        Maria|   Finance|  3000|
|       Robert|     Sales|  4100|
|        James|     Sales|  3000|
|        James|     Sales|  3000|
+-------------+----------+------+

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|          Jen|   Finance|  3900|
|        Maria|   Finance|  3000|
|        Scott|   Finance|  3000|
|        Maria|   Finance|  3000|
|       Robe

In [74]:
# p.98 sql쿼리처리->ViewFrame을 생성
df.createOrReplaceTempView("ViewFrame")
spark.sql("select department, salary from ViewFrame ORDER BY department asc").show()

+----------+------+
|department|salary|
+----------+------+
|   Finance|  3000|
|   Finance|  3000|
|   Finance|  3900|
|   Finance|  3000|
|     Sales|  3000|
|     Sales|  4100|
|     Sales|  3000|
+----------+------+



In [75]:
# P.99 groupBy: 원하는 컬럼 그룹화, count:그룹화한 데이터의 개수
df.show()
df.groupBy("department").count().show()

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|        James|     Sales|  3000|
|        Scott|   Finance|  3000|
|          Jen|   Finance|  3900|
|        Maria|   Finance|  3000|
+-------------+----------+------+

+----------+-----+
|department|count|
+----------+-----+
|     Sales|    3|
|   Finance|    4|
+----------+-----+



In [76]:
# p.100 groupby연산:sum,min,max,avg,mean
df.groupBy("department").sum("salary").show()
df.groupBy("department").min("salary").show()
df.groupBy("department").max("salary").show()
df.groupBy("department").avg( "salary").show()
df.groupBy("department").mean( "salary") .show()

+----------+-----------+
|department|sum(salary)|
+----------+-----------+
|     Sales|      10100|
|   Finance|      12900|
+----------+-----------+

+----------+-----------+
|department|min(salary)|
+----------+-----------+
|     Sales|       3000|
|   Finance|       3000|
+----------+-----------+

+----------+-----------+
|department|max(salary)|
+----------+-----------+
|     Sales|       4100|
|   Finance|       3900|
+----------+-----------+

+----------+------------------+
|department|       avg(salary)|
+----------+------------------+
|     Sales|3366.6666666666665|
|   Finance|            3225.0|
+----------+------------------+

+----------+------------------+
|department|       avg(salary)|
+----------+------------------+
|     Sales|3366.6666666666665|
|   Finance|            3225.0|
+----------+------------------+



In [77]:
# p.101 groupBy: 다수의 컬럼 그룹화 가능
df.groupBy("employee_name","department").sum("salary").show()

+-------------+----------+-----------+
|employee_name|department|sum(salary)|
+-------------+----------+-----------+
|          Jen|   Finance|       3900|
|        James|     Sales|       6000|
|        Maria|   Finance|       6000|
|        Scott|   Finance|       3000|
|       Robert|     Sales|       4100|
+-------------+----------+-----------+



In [78]:
# p.102 Join 
# 데이터프레임 2개 생성
emp = [(1,"Smith",-1,"2018","10","M",3000),
(2,"Rose",1,"2010","20","M",4000),
(3,"Williams",1,"2010","10","M",1000),
(4,"Jones",2,"2005","10","F",2000),
(5,"Brown",2,"2010","40","",-1),
(6,"Brown",2,"2010","50","",-1)]
empColumns = ["emp_id","name","superior_emp_id","year_joined","emp_dept_id","gender","salary"]
empDF = spark.createDataFrame(data=emp, schema = empColumns)
empDF.printSchema()
empDF.show(truncate=False)

dept = [("Finance",10),("Marketing",20),("Sales",30),("IT",40)]
deptColumns = ["dept_name","dept_id"]
deptDF = spark.createDataFrame(data=dept, schema = deptColumns)
deptDF.printSchema()
deptDF.show(truncate=False)

root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- superior_emp_id: long (nullable = true)
 |-- year_joined: string (nullable = true)
 |-- emp_dept_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
|6     |Brown   |2              |2010       |50         |      |-1    |
+------+--------+---------------+-----------+-----------+------+-----

In [80]:
# p.103 ~ 106
# inner join
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"inner").show()

#full outer join
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"outer").show()
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"full").show()
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"fullouter").show()

#left outer join
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"left").show()
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"leftouter").show()

#right outer join
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"right").show()
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"rightouter").show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+-

In [82]:
# p.107
#left semi join : inner join과 유사
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"leftsemi").show()

#left anti join: left semi join과 반대
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"leftanti").show()

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|
|     3|Williams|              1|       2010|         10|     M|  1000|
|     4|   Jones|              2|       2005|         10|     F|  2000|
|     2|    Rose|              1|       2010|         20|     M|  4000|
|     5|   Brown|              2|       2010|         40|      |    -1|
+------+--------+---------------+-----------+-----------+------+------+

+------+-----+---------------+-----------+-----------+------+------+
|emp_id| name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+-----+---------------+-----------+-----------+------+------+
|     6|Brown|              2|       2010|         50|      |    -1|
+------+-----+---------------+-----------+-----------+------+------+



In [84]:
# p.108
#self join
empDF.show()
empDF.alias("emp1").join(empDF.alias("emp2"), 
col("emp1.superior_emp_id") == col("emp2.emp_id"),"inner").select(col("emp1.emp_id"),col("emp1.name"),
col("emp2.emp_id").alias("superior_emp_id"), col("emp2.name").alias("superior_emp_name")).show()

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|
|     2|    Rose|              1|       2010|         20|     M|  4000|
|     3|Williams|              1|       2010|         10|     M|  1000|
|     4|   Jones|              2|       2005|         10|     F|  2000|
|     5|   Brown|              2|       2010|         40|      |    -1|
|     6|   Brown|              2|       2010|         50|      |    -1|
+------+--------+---------------+-----------+-----------+------+------+

+------+--------+---------------+-----------------+
|emp_id|    name|superior_emp_id|superior_emp_name|
+------+--------+---------------+-----------------+
|     2|    Rose|              1|            Smith|
|     3|Williams|              1|            Smith|
|     4|   Jones|  

In [86]:
# p.109 Union
# 데이터프레임 2개 생성
simpleData = [("A","B","C"), ("A","B","D"), ("A","B","E"), ("A","B","F"), ("A","B","D")]
columns = ["col_1","col_2","col_3"]
df = spark.createDataFrame(data = simpleData, schema = columns)
df.printSchema()
df.show(truncate=False)

simpleData2 = [("A","B","G"), ("A","B","H"), ("A","B","I"), ("A","B","F"), ("A","B","D")]
columns2 = ["col_1","col_2","col_3"]
df2 = spark.createDataFrame(data = simpleData2, schema = columns2)
df2.printSchema()
df2.show(truncate=False)

root
 |-- col_1: string (nullable = true)
 |-- col_2: string (nullable = true)
 |-- col_3: string (nullable = true)

+-----+-----+-----+
|col_1|col_2|col_3|
+-----+-----+-----+
|A    |B    |C    |
|A    |B    |D    |
|A    |B    |E    |
|A    |B    |F    |
|A    |B    |D    |
+-----+-----+-----+

root
 |-- col_1: string (nullable = true)
 |-- col_2: string (nullable = true)
 |-- col_3: string (nullable = true)

+-----+-----+-----+
|col_1|col_2|col_3|
+-----+-----+-----+
|A    |B    |G    |
|A    |B    |H    |
|A    |B    |I    |
|A    |B    |F    |
|A    |B    |D    |
+-----+-----+-----+



In [87]:
# p.110 Union: 합병
unionDF = df.union(df2)
unionDF.show()

disDF = df.union(df2).distinct()
disDF.show()

+-----+-----+-----+
|col_1|col_2|col_3|
+-----+-----+-----+
|    A|    B|    C|
|    A|    B|    D|
|    A|    B|    E|
|    A|    B|    F|
|    A|    B|    D|
|    A|    B|    G|
|    A|    B|    H|
|    A|    B|    I|
|    A|    B|    F|
|    A|    B|    D|
+-----+-----+-----+

+-----+-----+-----+
|col_1|col_2|col_3|
+-----+-----+-----+
|    A|    B|    E|
|    A|    B|    C|
|    A|    B|    D|
|    A|    B|    F|
|    A|    B|    H|
|    A|    B|    I|
|    A|    B|    G|
+-----+-----+-----+



In [88]:
# p.111 함수생성
def convertCase(str):
    resStr=""
    arr = str.split(" ")
    for x in arr:
       resStr= resStr + x[0:1].upper() + x[1:len(x)] + " "
    return resStr

columns = ["Seqno","Name"]
data = [("1", "john jones"),("2", "tracey smith"),("3", "amy sanders")]
df = spark.createDataFrame(data,columns)
df.show()

+-----+------------+
|Seqno|        Name|
+-----+------------+
|    1|  john jones|
|    2|tracey smith|
|    3| amy sanders|
+-----+------------+



In [89]:
# p.112 작성한 함수는 바로 사용할 수 없고 Pyspark의 udf 함수를 우선 적용하여야함
from pyspark.sql.functions import udf
convertUDF = udf(lambda z: convertCase(z))
df.select(col("Seqno"),convertUDF(col("Name")).alias("Name") ).show()

+-----+-------------+
|Seqno|         Name|
+-----+-------------+
|    1|  John Jones |
|    2|Tracey Smith |
|    3| Amy Sanders |
+-----+-------------+



In [91]:
# p.113
# map(): RDD 각 요소에 함수를 적용
# flatmap(): map으로 반환된 결과의 각 요소를 반환, 한번에 처리
rdd = sc.parallelize(['A,B,C','D,E,F','G,H,I'])

map_rdd = rdd.map(lambda x: x.split(','))
print(map_rdd.collect())

flatmap_rdd = rdd.flatMap(lambda x: x.split(','))
print(flatmap_rdd.collect())

[['A', 'B', 'C'], ['D', 'E', 'F'], ['G', 'H', 'I']]
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']


In [98]:
# p.114
# map:하나의 행이 하나의 튜플로 구성
# flatmap: 모든행이 한번에 처리
data = [('James','Smith','M',30),('Anna','Rose','F',41),('Robert','Williams','M',62)]
columns = ["firstname","lastname","gender","salary"]
df = spark.createDataFrame(data=data, schema = columns)
df.show()

rdd2=df.rdd.map(lambda x: (x[0] + "," + x[1], x[2], x[3]*2))
print(rdd2.collect())

rdd3=df.rdd.flatMap(lambda x: (x[0] + "," + x[1], x[2], x[3]*2))  
print(rdd3.collect())

+---------+--------+------+------+
|firstname|lastname|gender|salary|
+---------+--------+------+------+
|    James|   Smith|     M|    30|
|     Anna|    Rose|     F|    41|
|   Robert|Williams|     M|    62|
+---------+--------+------+------+

[('James,Smith', 'M', 60), ('Anna,Rose', 'F', 82), ('Robert,Williams', 'M', 124)]
['James,Smith', 'M', 60, 'Anna,Rose', 'F', 82, 'Robert,Williams', 'M', 124]


In [99]:
# p.115 sample: 무작위 추출 / 0~1 = 0~100%
df=spark.range(20)
sample_df = df.sample(0.2)
sample_df.show()

+---+
| id|
+---+
|  4|
| 14|
| 18|
+---+



In [100]:
df=spark.range(20)
sample_df = df.sample(0.2)
sample_df.show()

+---+
| id|
+---+
|  4|
| 19|
+---+



In [101]:
# p.116 시드값 추가하여 샘플링 데이터 기록
df.sample(0.2,123).show()
df.sample(0.2,123).show()
df.sample(0.2,456).show()
df.sample(0.2,456).show()

+---+
| id|
+---+
|  0|
|  4|
| 19|
+---+

+---+
| id|
+---+
|  0|
|  4|
| 19|
+---+

+---+
| id|
+---+
|  0|
|  1|
| 10|
| 19|
+---+

+---+
| id|
+---+
|  0|
|  1|
| 10|
| 19|
+---+



In [103]:
# p.117 True 입력시 중복 허용하여 샘플링
df.sample(True,0.2).show()
df.sample(0.2).show()

+---+
| id|
+---+
|  0|
| 10|
| 14|
| 14|
| 15|
+---+

+---+
| id|
+---+
|  0|
| 13|
+---+



In [107]:
# p.118 fill(): Dataframe 데이터가 0 등일때 전처리를 할 수 있는 기능
# csv파일 로드
df = spark.read.options(header='true', inferSchema='true').csv("small_zipcode.csv")
df.printSchema()
df.show(truncate=False)

root
 |-- id: integer (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- population: integer (nullable = true)

+---+-------+--------+-------------------+-----+----------+
|id |zipcode|type    |city               |state|population|
+---+-------+--------+-------------------+-----+----------+
|1  |704    |STANDARD|null               |PR   |30100     |
|2  |704    |null    |PASEO COSTA DEL SUR|PR   |null      |
|3  |709    |null    |BDA SAN LUIS       |PR   |3700      |
|4  |76166  |UNIQUE  |CINGULAR WIRELESS  |TX   |84000     |
|5  |76177  |STANDARD|null               |TX   |null      |
+---+-------+--------+-------------------+-----+----------+



In [108]:
# p.119 fill함수 내에 값들을 0으로 지정하면 기존의 null값을 0으로 변경
# subset: 지정컬럼의 데이터만 변경
df.na.fill(value=0).show()
df.na.fill(value=0,subset=["population"]).show()

+---+-------+--------+-------------------+-----+----------+
| id|zipcode|    type|               city|state|population|
+---+-------+--------+-------------------+-----+----------+
|  1|    704|STANDARD|               null|   PR|     30100|
|  2|    704|    null|PASEO COSTA DEL SUR|   PR|         0|
|  3|    709|    null|       BDA SAN LUIS|   PR|      3700|
|  4|  76166|  UNIQUE|  CINGULAR WIRELESS|   TX|     84000|
|  5|  76177|STANDARD|               null|   TX|         0|
+---+-------+--------+-------------------+-----+----------+

+---+-------+--------+-------------------+-----+----------+
| id|zipcode|    type|               city|state|population|
+---+-------+--------+-------------------+-----+----------+
|  1|    704|STANDARD|               null|   PR|     30100|
|  2|    704|    null|PASEO COSTA DEL SUR|   PR|         0|
|  3|    709|    null|       BDA SAN LUIS|   PR|      3700|
|  4|  76166|  UNIQUE|  CINGULAR WIRELESS|   TX|     84000|
|  5|  76177|STANDARD|               nu

In [112]:
# p.120 fill: 특정 문자열로 변경(빈문자열""로도 가능)
df.na.fill("unknown",["city"]).na.fill("",["type"]).show()
# json 타입의 문서형을 입력값으로 설정
df.na.fill({"city": "unknown", "type": ""}).show()

+---+-------+--------+-------------------+-----+----------+
| id|zipcode|    type|               city|state|population|
+---+-------+--------+-------------------+-----+----------+
|  1|    704|STANDARD|            unknown|   PR|     30100|
|  2|    704|        |PASEO COSTA DEL SUR|   PR|      null|
|  3|    709|        |       BDA SAN LUIS|   PR|      3700|
|  4|  76166|  UNIQUE|  CINGULAR WIRELESS|   TX|     84000|
|  5|  76177|STANDARD|            unknown|   TX|      null|
+---+-------+--------+-------------------+-----+----------+

+---+-------+--------+-------------------+-----+----------+
| id|zipcode|    type|               city|state|population|
+---+-------+--------+-------------------+-----+----------+
|  1|    704|STANDARD|            unknown|   PR|     30100|
|  2|    704|        |PASEO COSTA DEL SUR|   PR|      null|
|  3|    709|        |       BDA SAN LUIS|   PR|      3700|
|  4|  76166|  UNIQUE|  CINGULAR WIRELESS|   TX|     84000|
|  5|  76177|STANDARD|            unkno

In [113]:
# p.121
data = [("Banana",1000,"USA"), ("Carrots",1500,"USA"), ("Beans",1600,"USA"),
      ("Banana",2000,"Canada"),("Carrots",2000,"Canada"),("Beans",2000,"Mexico")]
columns= ["Product","Amount","Country"]
df = spark.createDataFrame(data = data, schema = columns)
df.printSchema()
df.show(truncate=False)

root
 |-- Product: string (nullable = true)
 |-- Amount: long (nullable = true)
 |-- Country: string (nullable = true)

+-------+------+-------+
|Product|Amount|Country|
+-------+------+-------+
|Banana |1000  |USA    |
|Carrots|1500  |USA    |
|Beans  |1600  |USA    |
|Banana |2000  |Canada |
|Carrots|2000  |Canada |
|Beans  |2000  |Mexico |
+-------+------+-------+



In [116]:
# p.122 Pivot: groupBy 그룹화를 지정하고 pivot 함수 사용
pivotDF = df.groupBy("Product").pivot("Country").sum("Amount")
pivotDF.printSchema()
pivotDF.show(truncate=False)

root
 |-- Product: string (nullable = true)
 |-- Canada: long (nullable = true)
 |-- Mexico: long (nullable = true)
 |-- USA: long (nullable = true)

+-------+------+------+----+
|Product|Canada|Mexico|USA |
+-------+------+------+----+
|Beans  |null  |2000  |1600|
|Banana |2000  |null  |1000|
|Carrots|2000  |null  |1500|
+-------+------+------+----+



In [117]:
# p.123 ArrayType: 배열 형태 데이터가 입력되고자 하는 컬럼에 적용
data = [("James,,Smith",["Java","Scala","C++"],["Spark","Java"],"OH","CA")]
from pyspark.sql.types import StringType, ArrayType,StructType,StructField
schema = StructType([ 
    StructField("name",StringType(),True), 
    StructField("languagesAtSchool",ArrayType(StringType()),True), 
    StructField("languagesAtWork",ArrayType(StringType()),True), 
    StructField("currentState", StringType(), True), 
    StructField("previousState", StringType(), True)
])
df = spark.createDataFrame(data=data,schema=schema)
df.printSchema()
df.show()

root
 |-- name: string (nullable = true)
 |-- languagesAtSchool: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- languagesAtWork: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- currentState: string (nullable = true)
 |-- previousState: string (nullable = true)

+------------+------------------+---------------+------------+-------------+
|        name| languagesAtSchool|languagesAtWork|currentState|previousState|
+------------+------------------+---------------+------------+-------------+
|James,,Smith|[Java, Scala, C++]|  [Spark, Java]|          OH|           CA|
+------------+------------------+---------------+------------+-------------+



In [118]:
# p.124 explode: 배열로 이루어진 컬럼에 적용하여 배열의 데이터를 풀어서 출력
from pyspark.sql.functions import explode
df.select(df.name,explode(df.languagesAtSchool)).show()
df.show()

+------------+-----+
|        name|  col|
+------------+-----+
|James,,Smith| Java|
|James,,Smith|Scala|
|James,,Smith|  C++|
+------------+-----+

+------------+------------------+---------------+------------+-------------+
|        name| languagesAtSchool|languagesAtWork|currentState|previousState|
+------------+------------------+---------------+------------+-------------+
|James,,Smith|[Java, Scala, C++]|  [Spark, Java]|          OH|           CA|
+------------+------------------+---------------+------------+-------------+



In [119]:
# p.125 split:특정기호로 이루어진 데이터를 배열 형태로 구성
from pyspark.sql.functions import split
df.select(split(df.name,",").alias("nameAsArray")).show()
df.show()

+----------------+
|     nameAsArray|
+----------------+
|[James, , Smith]|
+----------------+

+------------+------------------+---------------+------------+-------------+
|        name| languagesAtSchool|languagesAtWork|currentState|previousState|
+------------+------------------+---------------+------------+-------------+
|James,,Smith|[Java, Scala, C++]|  [Spark, Java]|          OH|           CA|
+------------+------------------+---------------+------------+-------------+



In [120]:
# p.126 array: 다수의 열에 있는 데이터 병합하여 새로운 배열 형태의 컬럼 생성
from pyspark.sql.functions import array
df.select(df.name,array(df.currentState,df.previousState).alias("States")).show()
df.show()

+------------+--------+
|        name|  States|
+------------+--------+
|James,,Smith|[OH, CA]|
+------------+--------+

+------------+------------------+---------------+------------+-------------+
|        name| languagesAtSchool|languagesAtWork|currentState|previousState|
+------------+------------------+---------------+------------+-------------+
|James,,Smith|[Java, Scala, C++]|  [Spark, Java]|          OH|           CA|
+------------+------------------+---------------+------------+-------------+



In [121]:
# p.127 array_contains: 특정 컬럼의 값이 있다면, 해당 컬럼의 데이터를 Boolean(True,False)로 표현 가능
from pyspark.sql.functions import array_contains
df.select(df.name,array_contains(df.languagesAtSchool,"Java").alias("array_contains")).show()
df.show()

+------------+--------------+
|        name|array_contains|
+------------+--------------+
|James,,Smith|          true|
+------------+--------------+

+------------+------------------+---------------+------------+-------------+
|        name| languagesAtSchool|languagesAtWork|currentState|previousState|
+------------+------------------+---------------+------------+-------------+
|James,,Smith|[Java, Scala, C++]|  [Spark, Java]|          OH|           CA|
+------------+------------------+---------------+------------+-------------+

